- 환경 변수 가져오기

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_KEY

'pcsk_6bYEch_C6MyWm9ftZgVbruvc9Tb4j8Ssd8SWZLhB5DYBTEG5orqgMPxZkh5SQ71Uew6fNe'

- Pinecone 클라이언트 초기화

In [3]:
from pinecone import  Pinecone,ServerlessSpec
pine = Pinecone(api_key=PINECONE_API_KEY)
# pine = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

- 인덱스 생성(서버리스)

In [4]:
index_name='textbook'

In [5]:
pine.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud='aws',region='us-east-1')
)

{
    "name": "textbook",
    "metric": "cosine",
    "host": "textbook-87mf77p.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

- 임베딩 객체 생성

In [6]:
# 사용할 인덱스 가져오기
index = pine.Index(index_name)
index.describe_index_stats()

c:\Users\C113\miniconda3\envs\llmEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [7]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
# 임베딩 모델의 차원(dimension) 과 생성한 인덱스의 dimension 이 같아야 합니다.

- 벡터DB 에 저장할 데이터셋 가져오기
  + 영어로 20231101 버전의 위키백과 600만개의 row 중 100개 만 가져오기
  + 데이터의 크기가 크면 임베딩 시간이 오래 걸림.

In [8]:
from datasets import load_dataset
# pip install datasets
# dataset = load_dataset("wikimedia/wikipedia","20231101.en",split="train[:100]")  # 7GB 전체를 다운로드
dataset = load_dataset("parquet",data_files=["train-04.parquet"])

Generating train split: 98996 examples [00:00, 124214.52 examples/s]


In [9]:
print(dataset)   # num_rows : 15만개

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 98996
    })
})


In [10]:
data = dataset['train'][0:100]    # 여전히 dict
print(len(data))   # dict : 4개의 키값.4개의 키 안에 리스트 크기 100
print(data['text'][0])
# select() 는 dataset 함수
data = dataset['train'].select(range(100))    # dict 의 리스트
print(len(data))   # list 의 크기 : 100

1
***사유지 담보대출의 개요***

사유지 담보대출은 개인이 자신의 사유지를 담보로 대출을 받는 것을 말합니다. 대출받는 금액은 일반적으로 사유지의 시세에 따라 결정되며, 대출 기간은 1년에서 30년까지 다양합니다. 사유지 담보대출은 주택 담보대출과 유사하지만, 주택 담보대출은 주거용 부동산을 담보로 받는 대출인 반면, 사유지 담보대출은 주거용 부동산을 제외한 모든 사유지를 담보로 받는 대출입니다.

***사유지 담보대출의 이점***

사유지 담보대출은 개인에게 다음과 같은 이점을 제공합니다.

* **자금 조달의 용이성:** 개인은 부동산에 대한 경제적 이익을 양도하지 않고도 사유지를 담보로 대출을 받을 수 있습니다.
* **낮은 금리:** 사유지 담보대출의 금리는 일반적으로 주택 담보대출의 금리보다 낮습니다.
* **신용 점수 향상:** 사유지 담보대출을 정기적으로 상환하면 신용 점수가 향상될 수 있습니다.
* **세금 혜택:** 사유지 담보대출의 이자는 세금 공제의 대상이 될 수 있습니다.

***사유지 담보대출의 한계***

사유지 담보대출은 개인에게 다음과 같은 한계를 가지고 있습니다.

* **담보 제공의 필요성:** 사유지 담보대출을 받기 위해서는 사유지를 담보로 제공해야 합니다.
* **대출 금액 제한:** 대출받는 금액은 일반적으로 사유지의 시세에 따라 결정되므로 필요한 금액을 모두 대출받지 못할 수 있습니다.
* **대출 기간 제한:** 대출 기간은 일반적으로 1년에서 30년까지 제한되어 있습니다.
* **위험성:** 사유지 담보대출은 담보가 제공되는 대출이므로 담보물의 가치가 하락하면 대출받은 금액을 상환하기 어려울 수 있습니다.

***사유지 담보대출 신청 절차***

사유지 담보대출을 신청하려면 다음과 같은 절차를 따르해야 합니다.

1. 금융기관에 사유지 담보대출 신청서를 제출합니다.
2. 금융기관이 신청서를 검토하고 대출 승인 여부를 결정합니다.
3. 대출 승인이 되면 대출 계약을 체결합니다.
4. 담보물에 대한 담보권을 설정합니

In [11]:
# print(data['text'][0])
print(data[50])

{'text': '# <ins> 반사는 모든 것이 그것이 볼 수 있는 것을 볼 수 있는 세계로 만드는 과정</ins>\n\n 여러분, 오늘 강의에서는 반사를 탐구하려고 합니다. 반사는 세상을 만들어 주는 자연의 중요한 과정입니다. \n\n> <ins>1. 세계와 그것의 모든 부분은 구제 불능적인 문제와 그 해결책으로 구성되어 있습니다.</ins>\n\n 모든 것은 그것이 볼 수 있는 것, 그것은 반영하는 것들로부터만 알 수 있습니다.\n\n> <ins>2. 모든 것은 그것이 볼 수 있는 것, 그것은 반영하는 것들로부터만 알 수 있습니다.</ins>\n\n 우리가 볼 수 있는 모든 것은 우리의 빛을 반사합니다.\n\n> <ins>3. 우리가 볼 수 있는 모든 것은 우리의 빛을 반사합니다.</ins>\n\n 반사의 재미있는 점은 그것이 모두 서로를 비추고 반사한다는 것입니다.\n\n> <ins>4. 반사의 재미있는 점은 그것이 모두 서로를 비추고 반사한다는 것입니다.</ins>\n\n 왜냐하면 모두가 서로를 볼 수 있도록 만들기 때문입니다.\n\n> <ins>5. 왜냐하면 모두가 서로를 볼 수 있도록 만들기 때문입니다.</ins>\n\n 서로를 비추고 반사하는 무언가가 없다면, 어둠과 암흑만이 존재했을 것입니다.\n\n> <ins>6. 서로를 비추고 반사하는 무언가가 없다면, 어둠과 암흑만이 존재했을 것입니다.</ins>\n\n 빛은 식물이 살아 있는 유일한 방법이며, 우리도 반사해야 합니다.\n\n> <ins>7. 빛은 식물이 살아 있는 유일한 방법이며, 우리도 반사해야 합니다.</ins>\n\n 우리는 우리가 좋아하는 것을 다른 사람들을 향해 반사했을 때 가장 행복합니다.\n\n> <ins>8. 우리는 우리가 좋아하는 것을 다른 사람들을 향해 반사했을 때 가장 행복합니다.</ins>\n\n 우리가 우리 자신과 다른 모든 것을 비추고 반사할 때 우리는 우리의 진정한 빛을 발견하기 시작합니다.\n\n> <ins>9. 우리가 우리 자신과 다른 모든 것을 비추고 반사

- 청크
  + splitter 객체를 생성해서 문자열을 나누기

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,     # 텍스트 분할 크기
    chunk_overlap=20,   # 분할할 텍스트의 중첩 크기
    length_function=len,
    separators = ['\n',' ']   # 분할할때 사용할 텍스트(단어) 구분자
)

- 청킹 후 임베딩 -> 업서트 : batch_size 크기 만큼 수행

In [16]:
from uuid import uuid4
import time

batch_size = 30
texts = []
metas = []
count = 0

# 데이터셋의 각 샘플에 대해 반복합니다.  features: ['id', 'url', 'title', 'text'],
for i,sample in enumerate(data):
    full_text = sample["text"] # Wikipedia 문서 텍스트 -> 청킹 후 임베딩
    metadata = {  # key 구성은 임의로 합니다. 청킹된 데이터의 소속을 구별
        'text': str(sample["text"]),  # Wikipedia 문서 ID
    }


    chunks = splitter.split_text(full_text)  # 텍스트를 청크로 분할합니다.
    print(len(chunks))  # full_text 를 몇개로 분할 했는지 확인

    # 각 청크에 대해 반복합니다.
    for i, chunk in enumerate(chunks):
        # 실제로 벡터db에 업서트할 record
        record = {
            'chunk_id': i,  # 청크 ID
            'head_chunk': chunk[:200],  # 전체 텍스트
            **metadata,  # 메타데이터 언패킹
        }

        texts.append(chunk)  # 청크(분할된 텍스트)를 텍스트 목록에 추가합니다.
        metas.append(record)  # 메타데이터를 메타데이터 목록에 추가합니다.

        count += 1  # 처리된 청크 수를 증가시킵니다.

        # batch_size만큼의 청크를 처리 : 임베딩 -> 업서트
        if count % batch_size == 0:
            # Pinecone 인덱스에 청크를 추가합니다.청크 text 갯수(batch_size) 만큼 레코드 uuid 만들기
            ids = [str(uuid4()) for _ in range(len(texts))]
            # 임베딩. 30개씩(batch_size).
            embeddings = embedding.embed_documents(texts)
            index.upsert(
                vectors=zip(ids, embeddings, metas),
                namespace="textbook-ns1")
            # 청크 목록과 메타데이터 목록을 비웁니다.
            texts = []
            metas = []
            # 1초 대기합니다.
            time.sleep(1)

5
7
3
5
9
6
8
5
3
5
4
7
7
4
5
8
7
5
5
7
6
6
6
9
3
5
4
4
4
3
4
4
7
5
6
5
10
4
8
4
5
5
4
8
5
9
14
7
5
9
4
5
10
7
10
4
4
5
6
4
7
6
4
8
4
4
3
5
5
10
3
3
5
8
4
16
5
9
7
5
4
8
9
5
6
5
8
8
4
10
4
10
5
6
5
8
3
5
13
6
